In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Using User Defined Functions

In [ ]:
from numpy import NaN
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DoubleType

udf1 = UserDefinedFunction(lambda x: NaN if x > 0.5 else x, DoubleType())
udf2 = UserDefinedFunction(lambda x: NaN if x > 1.0 else x, DoubleType())

In [ ]:
from pyspark.sql.functions import rand, randn

df = sqlc.range(0, 5).withColumn("uniform", rand(7)).withColumn("normal", randn(7))

dfnan = df.withColumn("nanUniform", udf1("uniform")) \
          .withColumn("nanNormal", udf2("normal")) \
          .drop("uniform").withColumnRenamed("nanUniform", "uniform") \
          .drop("normal").withColumnRenamed("nanNormal", "normal")

In [ ]:
dfnan.show()

## NA Functions

### Drop

In [ ]:
dfnan.na.drop(how='all',subset=['uniform','normal']).show()

### Replace

In [ ]:
dfnan.na.replace([NaN],[0.0], 'uniform').show()

### Fill

In [ ]:
dfnan.na.fill({'uniform': 0.0, 'normal': 1.0}).show()

In [ ]:
dfsummary = dfnan.drop("id").na.drop(how='any').describe()
dfsummary.show()

In [ ]:
uniformMean = float(dfsummary.rdd.collect()[1][1])
print(uniformMean)

In [ ]:
pandasSummary = dfsummary.toPandas().set_index("summary")
pandasSummary

In [ ]:
uniformMean = float(pandasSummary.loc['mean'].uniform)
print(uniformMean)

In [ ]:
normalMean = float(pandasSummary.loc['mean'].normal)
print(normalMean)

In [ ]:
dfnan.na.fill({"uniform": uniformMean, "normal": normalMean}).show()

## Duplicates

In [ ]:
dfDuplicates = dfnan.union(sc.parallelize([(5,1,1),(6,1,1)]).toDF())

In [ ]:
dfDuplicates.show()

In [ ]:
dfDuplicates.dropDuplicates(["uniform","normal"]).show()

In [ ]:
sc.stop()